In [2]:
#Include:
# Gestione in chunks
# Gestione esplicita colonne da ingestionare
# Gestione esplictita tipi colonne + Validazione dello schema
# Consolidamento Parquet

from concurrent.futures import ThreadPoolExecutor, as_completed
import pyreadstat
from datetime import datetime
import sys
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import shutil
import time
import os
import gc
import glob

colspecs = [['BANCA' , '$5.'],['DT_RIFERIMENTO' , 'DATE9.'],['NDG' , '$16.'],['SPORTELLO' , '6'],['TIPO_RAP' , '$2.'],['NUM_CONTO' , '16'],['PROGR_FIDO' , '4'],['FT_FIDO' , '$5.'],['CREDIT_LINE' , '$1.'],['FLAG_RIPARTO' , '$1.'],['PRIORITA' , '4'],['FIDO_1' , '20,2'],['SALDO_CONT_1' , '20,2'],['SCONF_1' , '20,2'],['MARGINE_1' , '20,2'],['SALDO_DISP_1' , '20,2'],['UTILIZZO_1' , '20,2'],['FIDO_2' , '20,2'],['SALDO_CONT_2' , '20,2'],['SCONF_2' , '20,2'],['MARGINE_2' , '20,2'],['SALDO_DISP_2' , '20,2'],['UTILIZZO_2' , '20,2'],['FIDO_3' , '20,2'],['SALDO_CONT_3' , '20,2'],['SCONF_3' , '20,2'],['MARGINE_3' , '20,2'],['SALDO_DISP_3' , '20,2'],['UTILIZZO_3' , '20,2'],['FIDO_4' , '20,2'],['SALDO_CONT_4' , '20,2'],['SCONF_4' , '20,2'],['MARGINE_4' , '20,2'],['SALDO_DISP_4' , '20,2'],['UTILIZZO_4' , '20,2'],['FIDO_5' , '20,2'],['SALDO_CONT_5' , '20,2'],['SCONF_5' , '20,2'],['MARGINE_5' , '20,2'],['SALDO_DISP_5' , '20,2'],['UTILIZZO_5' , '20,2'],['FIDO_6' , '20,2'],['SALDO_CONT_6' , '20,2'],['SCONF_6' , '20,2'],['MARGINE_6' , '20,2'],['SALDO_DISP_6' , '20,2'],['UTILIZZO_6' , '20,2'],['FIDO_7' , '20,2'],['SALDO_CONT_7' , '20,2'],['SCONF_7' , '20,2'],['MARGINE_7' , '20,2'],['SALDO_DISP_7' , '20,2'],['UTILIZZO_7' , '20,2'],['FIDO_8' , '20,2'],['SALDO_CONT_8' , '20,2'],['SCONF_8' , '20,2'],['MARGINE_8' , '20,2'],['SALDO_DISP_8' , '20,2'],['UTILIZZO_8' , '20,2'],['FIDO_9' , '20,2'],['SALDO_CONT_9' , '20,2'],['SCONF_9' , '20,2'],['MARGINE_9' , '20,2'],['SALDO_DISP_9' , '20,2'],['UTILIZZO_9' , '20,2'],['FIDO_10' , '20,2'],['SALDO_CONT_10' , '20,2'],['SCONF_10' , '20,2'],['MARGINE_10' , '20,2'],['SALDO_DISP_10' , '20,2'],['UTILIZZO_10' , '20,2'],['FIDO_11' , '20,2'],['SALDO_CONT_11' , '20,2'],['SCONF_11' , '20,2'],['MARGINE_11' , '20,2'],['SALDO_DISP_11' , '20,2'],['UTILIZZO_11' , '20,2'],['FIDO_12' , '20,2'],['SALDO_CONT_12' , '20,2'],['SCONF_12' , '20,2'],['MARGINE_12' , '20,2'],['SALDO_DISP_12' , '20,2'],['UTILIZZO_12' , '20,2'],['FIDO_13' , '20,2'],['SALDO_CONT_13' , '20,2'],['SCONF_13' , '20,2'],['MARGINE_13' , '20,2'],['SALDO_DISP_13' , '20,2'],['UTILIZZO_13' , '20,2'],['FIDO_14' , '20,2'],['SALDO_CONT_14' , '20,2'],['SCONF_14' , '20,2'],['MARGINE_14' , '20,2'],['SALDO_DISP_14' , '20,2'],['UTILIZZO_14' , '20,2'],['FIDO_15' , '20,2'],['SALDO_CONT_15' , '20,2'],['SCONF_15' , '20,2'],['MARGINE_15' , '20,2'],['SALDO_DISP_15' , '20,2'],['UTILIZZO_15' , '20,2'],['FIDO_16' , '20,2'],['SALDO_CONT_16' , '20,2'],['SCONF_16' , '20,2'],['MARGINE_16' , '20,2'],['SALDO_DISP_16' , '20,2'],['UTILIZZO_16' , '20,2'],['FIDO_17' , '20,2'],['SALDO_CONT_17' , '20,2'],['SCONF_17' , '20,2'],['MARGINE_17' , '20,2'],['SALDO_DISP_17' , '20,2'],['UTILIZZO_17' , '20,2'],['FIDO_18' , '20,2'],['SALDO_CONT_18' , '20,2'],['SCONF_18' , '20,2'],['MARGINE_18' , '20,2'],['SALDO_DISP_18' , '20,2'],['UTILIZZO_18' , '20,2'],['FIDO_19' , '20,2'],['SALDO_CONT_19' , '20,2'],['SCONF_19' , '20,2'],['MARGINE_19' , '20,2'],['SALDO_DISP_19' , '20,2'],['UTILIZZO_19' , '20,2'],['FIDO_20' , '20,2'],['SALDO_CONT_20' , '20,2'],['SCONF_20' , '20,2'],['MARGINE_20' , '20,2'],['SALDO_DISP_20' , '20,2'],['UTILIZZO_20' , '20,2'],['FIDO_21' , '20,2'],['SALDO_CONT_21' , '20,2'],['SCONF_21' , '20,2'],['MARGINE_21' , '20,2'],['SALDO_DISP_21' , '20,2'],['UTILIZZO_21' , '20,2'],['FIDO_22' , '20,2'],['SALDO_CONT_22' , '20,2'],['SCONF_22' , '20,2'],['MARGINE_22' , '20,2'],['SALDO_DISP_22' , '20,2'],['UTILIZZO_22' , '20,2'],['FIDO_23' , '20,2'],['SALDO_CONT_23' , '20,2'],['SCONF_23' , '20,2'],['MARGINE_23' , '20,2'],['SALDO_DISP_23' , '20,2'],['UTILIZZO_23' , '20,2'],['FIDO_24' , '20,2'],['SALDO_CONT_24' , '20,2'],['SCONF_24' , '20,2'],['MARGINE_24' , '20,2'],['SALDO_DISP_24' , '20,2'],['UTILIZZO_24' , '20,2'],['FIDO_25' , '20,2'],['SALDO_CONT_25' , '20,2'],['SCONF_25' , '20,2'],['MARGINE_25' , '20,2'],['SALDO_DISP_25' , '20,2'],['UTILIZZO_25' , '20,2'],['FIDO_26' , '20,2'],['SALDO_CONT_26' , '20,2'],['SCONF_26' , '20,2'],['MARGINE_26' , '20,2'],['SALDO_DISP_26' , '20,2'],['UTILIZZO_26' , '20,2'],['FIDO_27' , '20,2'],['SALDO_CONT_27' , '20,2'],['SCONF_27' , '20,2'],['MARGINE_27' , '20,2'],['SALDO_DISP_27' , '20,2'],['UTILIZZO_27' , '20,2'],['FIDO_28' , '20,2'],['SALDO_CONT_28' , '20,2'],['SCONF_28' , '20,2'],['MARGINE_28' , '20,2'],['SALDO_DISP_28' , '20,2'],['UTILIZZO_28' , '20,2'],['FIDO_29' , '20,2'],['SALDO_CONT_29' , '20,2'],['SCONF_29' , '20,2'],['MARGINE_29' , '20,2'],['SALDO_DISP_29' , '20,2'],['UTILIZZO_29' , '20,2'],['FIDO_30' , '20,2'],['SALDO_CONT_30' , '20,2'],['SCONF_30' , '20,2'],['MARGINE_30' , '20,2'],['SALDO_DISP_30' , '20,2'],['UTILIZZO_30' , '20,2'],['FIDO_31' , '20,2'],['SALDO_CONT_31' , '20,2'],['SCONF_31' , '20,2'],['MARGINE_31' , '20,2'],['SALDO_DISP_31' , '20,2'],['UTILIZZO_31' , '20,2'],['DT_ELABORAZIONE' , 'DATE9.']]


sas_file_path = r"C:\Users\EE01258\OneDrive - Unicredit\Desktop\45cols_202510.sas7bdat"
sas_file_base_name = os.path.basename(sas_file_path)

target_extension = ".parquet"
file_prefix = sas_file_base_name.replace(".sas7bdat","") 
table_folder = file_prefix
chunks_folder = os.path.join("chunks_folder", table_folder) #for Chunked Parquets
output_folder = os.path.join("output_folder", table_folder) #for consolidated Parquets

col_list = [col[0] for col in colspecs]

offset_days = 3653
offset_seconds = 315619200

chunksize = 1000000
offset = 0
chunk_idx = 0
max_workers = 1

def timenow():
    timenow = datetime.now().strftime('%H:%M:%S')
    return timenow

def map_sas_to_arrow(sas_format):

    if sas_format.startswith("$"):
        return pa.string()

    elif sas_format.upper().startswith("DATE"):
        return  pa.date32()

    elif sas_format.upper().startswith("DATETIME"):
        return pa.timestamp('s')

    elif "," in sas_format.upper():
        precision, scale = sas_format.split(",")
        return pa.decimal128(int(precision), int(scale))

    elif sas_format.isdigit():
        return pa.int64()

    else:
        raise ValueError(f"Tipo SAS non gestito: {sas_format}")

def build_schema(colspecs):
    
    # We need two schema:
    # the first one when we convert to Arrow from dictionaries
    # the second when we fix date/datetime columns (we do it later because is more performant than doing it during SAS Conversion)

    fields_bc = []
    fields_ac = []
    date_fields = []
    datetime_fields = []

    for column in colspecs:
        col_name,sas_format = column[0], column[1]
        col_type = map_sas_to_arrow(sas_format)
        
        if col_type == pa.date32():

            col_type_bc = pa.int64()
            col_type_ac = pa.date32()
            date_fields.append(col_name)

        if col_type == pa.timestamp('s'):

            col_type_bc = pa.int64()
            col_type_ac = pa.timestamp('s')
            datetime_fields.append(col_name)
        
        else: 
            col_type_bc = col_type
            col_type_ac = col_type

        fields_bc.append(pa.field(col_name,col_type_bc))
        fields_ac.append(pa.field(col_name,col_type_ac))

    arrow_schema_before_cast = pa.schema(fields_bc)
    arrow_schema_after_cast  = pa.schema(fields_ac)

    return arrow_schema_before_cast,arrow_schema_after_cast, date_fields, datetime_fields

def file_checker(chunks_folder, sas_file_path, chunksize):
         
    if os.path.exists(chunks_folder):
            shutil.rmtree(chunks_folder)
    os.mkdir(chunks_folder)

    #Controllo_Iniziale_File

    if os.path.exists(sas_file_path):
        _,meta = pyreadstat.read_sas7bdat(sas_file_path, metadataonly = True)
    else:
        print(f"File_Check: File {sas_file_path} non trovato")
        sys.exit(1)

    print("A1")
    rows = meta.number_rows
    num_chunks = (rows + chunksize - 1)//chunksize
    del _
    del meta

    print(f"File Trovato: #Rows: {rows}, chunksize: {chunksize}, #Chunks:{num_chunks}")
    return chunksize,num_chunks 


#Operator
def process_chunk(chunks_folder, file_prefix, chunk_idx, chunksize, arrow_schema_before_cast, arrow_schema_after_cast, date_fields, datetime_fields):

    if chunk_idx == 0: print("Chunk Processing: Start")
    filepath = os.path.join(chunks_folder, file_prefix + "#"  + str(chunk_idx) + ".parquet")

    chunk_start = time.time()
    offset = chunk_idx * chunksize
    
    try:
        chunk, _ = pyreadstat.read_sas7bdat(
            sas_file_path,
            row_offset= offset,
            row_limit= chunksize,
            output_format = "dict",
            encoding = "windows-1252",
            disable_datetime_conversion = True,
            usecols = col_list
            )
    except Exception as e:
        print(f"{chunk_idx}: Riga{chunk_idx + 1}: Errore critico in lettura: {e}")
        sys.exit(1)
    gc.collect()

    #create arrow table and delete dictionaries to free RAM
    try:
        table = pa.table(chunk, schema = arrow_schema_before_cast)
    except Exception as e:
        print(f"Conversion_to_Arrow: ERROR --> {e}")
        sys.exit(1)
    del chunk, _

    #offset Date and datetime columns, then cast
    casted_columns = []
    for field,col in zip(table.schema,table.columns):

        if field.name in date_fields:  
            adjusted_col = pc.subtract(col, pa.scalar(offset_days))
            casted_col = pc.cast(adjusted_col, pa.date32())
            casted_columns.append(casted_col)

        elif field.name in datetime_fields:
            adjusted_col = pc.subtract(col, pa.scalar(offset_seconds))
            casted_col = pc.cast(adjusted_col, pa.timestamp('s'))
            casted_columns.append(casted_col)       

        else: casted_columns.append(col)     
    
    casted_table = pa.Table.from_arrays (casted_columns, schema =  arrow_schema_after_cast)
    del table

    pq.write_table(casted_table, filepath, compression = 'zstd', compression_level = 1)
    del casted_table

    gc.collect()
    chunk_end = time.time()
    chunk_elab_time = round((chunk_end - chunk_start),0)

    print(f"Chunk {chunk_idx}: Completed ✅ | ElabTime: {chunk_elab_time} s | {timenow()}")

 
def consolidate_parquet(chunks_folder, output_folder, file_prefix):
    
    print("Consolidazione Parquet Finale")

    os.makedirs(output_folder, exist_ok=True)
    consolidation_start = time.time()

    files = sorted(glob.glob(os.path.join(chunks_folder, "*.parquet")),
                   key=lambda f: int(f.split("#")[-1].split(".")[0]))

    out_path = os.path.join(output_folder, f"{file_prefix}.parquet")
    writer = None
    try:
        for file in files:
            table = pq.read_table(file)
            if writer is None:
                writer = pq.ParquetWriter(out_path, table.schema, compression="zstd", compression_level=3)
            writer.write_table(table)
    finally:
        if writer:
            writer.close()

#Main with Iteration
def main():

    chunksize, num_chunks = file_checker(
        chunks_folder = chunks_folder,
        sas_file_path = sas_file_path,
        chunksize = chunksize
    )

    arrow_schema_before_cast, arrow_schema_after_cast, date_fields, datetime_fields = build_schema(colspecs = colspecs)

    with ThreadPoolExecutor(max_workers = max_workers) as executor:
            futures = {
                executor.submit(
                    process_chunk,
                    chunks_folder,
                    file_prefix, 
                    chunk_idx,
                    chunksize,
                    arrow_schema_before_cast,
                    arrow_schema_after_cast,
                    date_fields,
                    datetime_fields
                    ): chunk_idx for chunk_idx in range(num_chunks)
            }

    for future in as_completed(futures):
        idx = futures[future]
        try:
            future.result()
        except Exception as e:
            print(f"[Chunk {idx}] Producer error: {e}")

    consolidate_parquet(
         chunks_folder = chunks_folder,
         output_folder = output_folder,
         file_prefix = file_prefix
    )

main()

A-1
A0
A1
File Trovato: #Rows: 8979525, chunksize: 1000000, #Chunks:9
Chunk Processing: Start
Chunk 0: Completed ✅ | ElabTime: 24.0 s | 00:38:46
Chunk 1: Completed ✅ | ElabTime: 24.0 s | 00:39:10
Chunk 2: Completed ✅ | ElabTime: 25.0 s | 00:39:35
Chunk 3: Completed ✅ | ElabTime: 25.0 s | 00:40:00
Chunk 4: Completed ✅ | ElabTime: 28.0 s | 00:40:28
Chunk 5: Completed ✅ | ElabTime: 28.0 s | 00:40:55
Chunk 6: Completed ✅ | ElabTime: 35.0 s | 00:41:30
Chunk 7: Completed ✅ | ElabTime: 29.0 s | 00:41:59
Chunk 8: Completed ✅ | ElabTime: 29.0 s | 00:42:27
Consolidazione Parquet Finale
Consolidation: Terminated | ElabTime: 23.0


In [33]:
import time
import pyarrow.dataset as ds

input_folder = "output_folder_zstd"

start_time = time.time()

# Leggi tutti i file Parquet nella cartella come dataset
dataset = ds.dataset(input_folder, format="parquet")

# Carica l'intero dataset in memoria (solo per test)
table = dataset.to_table()

end_time = time.time()
elapsed = end_time - start_time



# Stampa informazioni
print("📄 Schema del dataset:")
print(f"\n📊 Numero di righe: {table.num_rows}")
print(f"📦 Numero di colonne: {table.num_columns}")
print("\n⏱️ Tempo di lettura: {:.3f} secondi".format(elapsed))

del table


📄 Schema del dataset:

📊 Numero di righe: 8560000
📦 Numero di colonne: 15

⏱️ Tempo di lettura: 0.861 secondi
